### Author: Carmen Alonso Martínez

# Web scraping

Cuando necesitamos extraer información publicada en internet, lo ideal es consultar una API, porque:

* Las respuestas contienen información estructurada
* En general, el propio servicio nos da documentación sobre cómo hacer peticiones y qué tipo de información podemos solicitar

Pero muchas veces nos encontramos con información en páginas web (en formato [HTML](https://es.wikipedia.org/wiki/HTML)) que nos gustaría obtener, pero sin API disponible.

Estas páginas `HTML` tienen cierta estructura, aunque con ciertos contras:

* Es más compleja, puede tener muchos niveles de anidamiento
* Es inestable. Están diseñadas para que se vean bien desde el explorador, no para guardar una estructura de consulta. De un día para otro, puede verse alterada por la incorporación de nuevos elementos visuales u otros motivos.
* Puede ser modificada por código cliente (javascript) en diferentes momentos: al cargar la página, al interaccionar con algún elemento, ...

### Ejercicio

Desde tu explorador, consulta el código fuente de una página de tu interés. Por ejemplo, para hacerlo en chrome:

* Accede a la página, p.e. [esta](https://es.wikipedia.org/wiki/HTML).
* Haz click derecho y pulsa `View page source`. Otra opción es pulsar `Inspect`, que además abrirá las herramientas de desarrollador de Chrome, muy útiles para navegar por la estructura de la página.

## Scraping de elementos html

La librería que vamos a utilizar es [Beautiful Soup](https://pypi.org/project/beautifulsoup4/). Nos permite buscar elementos y navegar por la estructura del html fácilmente.

Vemos dos ejemplos, uno sobre milanuncios y otro sobre spotahome, por si nos _banean_.

### milanuncios

Imaginemos que queremos comparar precios de un determinado modelo de motocicleta de segunda mano. P.e. con [esta búsqueda](https://www.milanuncios.com/motos-de-carretera/duke-390.htm) en milanuncios.

La mayor parte de las webs con contenido interesante (que hacen negocio gracias a su contenido) intentan protegerlas para evitar que les hagan scraping. Hay varias formas de simular que nuestro script es humano en lugar de un bot, algunas más básicas y otras más complejas. Por ahora, vamos a sobrescribir nuestro _user agent_. Es una cabecera que va en las peticiones diciendo quiénes somos (p.e. qué tipo de explorador usamos). Por defecto, la librería `requests` que vamos a usar, avisa que somos un bot. Vamos a sobrescribir esta cabecera para _disimular_ un poco. Podemos copiar uno popular de (aquí)[https://developers.whatismybrowser.com/useragents/explore/software_type_specific/web-browser/].

In [5]:
import requests
import bs4 # Para poder comprobar la versión hay que importar el paquete entero.
from bs4 import BeautifulSoup # Aunque realmente lo que nos interesa es esta parte del 
                              # paquete (una vez añadida la línea anterior, esta se podría haber quitado).

In [6]:
bs4.__version__

'4.9.1'

In [7]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
} # Definimos este objeto porque no tenemos claro que tipo de header pasa la librería de bs4 a la página cuando pedimos datos.
# Esto se usa para que no reconozca nuestra solicitud como la de un bot, en cuyo caso nos podrían no devolver datos.

Ahora, nos descargamos el html con `requests`.

In [8]:
page = requests.get('https://www.milanuncios.com/motos-de-carretera/duke-390.htm', headers=headers)
page


<Response [200]>

Podemos ver el contenido examinando la propiedad `content`.

In [5]:
page.content # Esto es todo lo que nos devuelve, pero esto no es manejable. Usamos BeautifulSoup para mejorar cómo se ve esto.

b'<!DOCTYPE html>\n<html xmlns="http://www.w3.org/1999/xhtml" xml:lang="es" lang="es">\n<head><title>MIL ANUNCIOS.COM - Duke 390. Motos de carretera de ocasion duke 390: Aprilia, BMW, Gagiva, Dervi, Honda, Yamaha, Kawasaki, Suzuki.</title>\n<meta http-equiv="Content-Type" content="text/html; charset=iso-8859-1">\n    <meta name="robots" content="noarchive">\n        <meta name="description" content="Compra venta de motos de carretera de ocasi\xf3n duke 390. Todas las marcas: Aprilia, BMW, Cagiva, Derbi, Honda, Yamaha, Kawasaki, Suzuki">\n<link rel="shortcut icon" href="https://static.milanuncios.com/202012151402-master/favicon.ico">    <link rel="stylesheet" type="text/css" href="https://static.milanuncios.com/202012151402-master/css/estilos.css">\n    <link rel="canonical" href="https://www.milanuncios.com/motos-de-carretera/duke-390.htm" />\n<script src="//c.dcdn.es/borostcf/plugins/BorosTcfAdevintaPlugin.pro.js"></script>\n<script src="//c.dcdn.es/openads/milanuncios/MilanunciosOpen

Este contenido es solo texto, no tiene estructura. Aún no podemos hacer búsquedas ni navegar por él.

Para hacerlo, creamos una instancia de `Beautiful Soup` y lo parseamos

In [10]:
soup = BeautifulSoup(page.content, 'html.parser') # Esto pone las cosas un poco más bonitas.

In [7]:
soup 

<!DOCTYPE html>

<html lang="es" xml:lang="es" xmlns="http://www.w3.org/1999/xhtml">
<head><title>MIL ANUNCIOS.COM - Duke 390. Motos de carretera de ocasion duke 390: Aprilia, BMW, Gagiva, Dervi, Honda, Yamaha, Kawasaki, Suzuki.</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="noarchive" name="robots"/>
<meta content="Compra venta de motos de carretera de ocasión duke 390. Todas las marcas: Aprilia, BMW, Cagiva, Derbi, Honda, Yamaha, Kawasaki, Suzuki" name="description"/>
<link href="https://static.milanuncios.com/202012151402-master/favicon.ico" rel="shortcut icon"/> <link href="https://static.milanuncios.com/202012151402-master/css/estilos.css" rel="stylesheet" type="text/css"/>
<link href="https://www.milanuncios.com/motos-de-carretera/duke-390.htm" rel="canonical">
<script src="//c.dcdn.es/borostcf/plugins/BorosTcfAdevintaPlugin.pro.js"></script>
<script defer="" src="//c.dcdn.es/openads/milanuncios/MilanunciosOpenAdsClient.pro.js"></scrip

#### SOS: ¡me han baneado!

Si estamos haciendo esto en clase, es muy probable que a la mayoría, nos detecten como bots y nos baneen. Esto pasa porque somos 30 o 40 personas, haciendo la misma petición desde la misma IP y con el mismo user agent.

Puedes usar el html offline que hay en `dat` para seguir con el ejercicio. Descomenta el siguiente código y sigue adelante

In [8]:
# Descomenta esto si te han baneado
# with open('dat/milanuncios.html') as f:
#     soup = BeautifulSoup(f, 'html.parser')

Sobre esto, podemos hacer búsquedas con `find` y `find_all` (o `select_one` y `select` si prefieres utilizar [selectores css](https://en.wikipedia.org/wiki/Cascading_Style_Sheets#Selector)). Sobre nuestro ejemplo, vamos a buscar todos los precios. Examinando el código fuente, vemos que son etiquetas `div` con clase `aditem-price`.

In [9]:
div_precios = soup.find_all('div', class_='aditem-price') # Para saber que tenemos coger <div> y esa clase, primero inspeccionamos
# los precios de idealista y miramos que, efectivamente, estas tabs y clase se usan en los precios normalmente. Vamos probando.
div_precios

# El objetivo al final es filtrar los tabs de html que contengan la información que nos interesa de la web. 
# Obsérvese que soup.find_all te devuelve una lista.

[<div class="aditem-price">4.800<sup>€</sup></div>,
 <div class="aditem-price">3.500<sup>€</sup></div>,
 <div class="aditem-price">4.500<sup>€</sup></div>,
 <div class="aditem-price">3.850<sup>€</sup></div>,
 <div class="aditem-price">1.000<sup>€</sup></div>,
 <div class="aditem-price">3.600<sup>€</sup></div>,
 <div class="aditem-price">2.900<sup>€</sup></div>,
 <div class="aditem-price">3.599<sup>€</sup></div>,
 <div class="aditem-price">4.700<sup>€</sup></div>,
 <div class="aditem-price">2.800<sup>€</sup></div>,
 <div class="aditem-price">3.400<sup>€</sup></div>,
 <div class="aditem-price">2.250<sup>€</sup></div>,
 <div class="aditem-price">3.850<sup>€</sup></div>,
 <div class="aditem-price">4.200<sup>€</sup></div>,
 <div class="aditem-price">3.000<sup>€</sup></div>,
 <div class="aditem-price">3.900<sup>€</sup></div>,
 <div class="aditem-price">2.900<sup>€</sup></div>,
 <div class="aditem-price">4.500<sup>€</sup></div>,
 <div class="aditem-price">3.400<sup>€</sup></div>,
 <div class=

`find_all` devuelve una __lista__ de elementos. Sobre ellos, podemos hacer:

`children` para sacar el listado de todos los hijos.

In [10]:
div_precios[0].children

In [11]:
list(div_precios[0].children) # children es un iterable (por eso hay que pasarlo a lista), 
# pero es lo que nos separa los valores que hemos extraído previamente con Beatifulsoup.

['4.800', <sup>€</sup>]

`get_text()` para sacar el texto de todos los hijos

In [12]:
div_precios[0].get_text() # Deja fuera los tabs como <sup>.

'4.800€'

Por tanto, para sacar el listado de todos los precios podemos hacer:

In [13]:
mi_lista=[]

for elemento in div_precios:

  mi_lista.append(list(elemento.children)[0])

mi_lista

['4.800',
 '3.500',
 '4.500',
 '3.850',
 '1.000',
 '3.600',
 '2.900',
 '3.599',
 '4.700',
 '2.800',
 '3.400',
 '2.250',
 '3.850',
 '4.200',
 '3.000',
 '3.900',
 '2.900',
 '4.500',
 '3.400',
 '3.000',
 '2.800',
 '4.900',
 '3.600',
 '2.850',
 '3.999',
 '3.500',
 '3.000',
 '4.200',
 '3.500',
 '3.399']

Esto último se puede reducir a la siguiente línea:

In [14]:
[list(div_precio.children)[0] for div_precio in div_precios]

['4.800',
 '3.500',
 '4.500',
 '3.850',
 '1.000',
 '3.600',
 '2.900',
 '3.599',
 '4.700',
 '2.800',
 '3.400',
 '2.250',
 '3.850',
 '4.200',
 '3.000',
 '3.900',
 '2.900',
 '4.500',
 '3.400',
 '3.000',
 '2.800',
 '4.900',
 '3.600',
 '2.850',
 '3.999',
 '3.500',
 '3.000',
 '4.200',
 '3.500',
 '3.399']

Tienes más funciones útiles con pequeños ejemplos [aquí](http://akul.me/blog/2016/beautifulsoup-cheatsheet/)

### Ejercicio

Crea un dataframe de pandas en el que cada fila sea un anuncio y tenga como columnas información que consideres relevante: precio, kilómetros, año, cilindrada, texto del anuncio, ...

In [15]:
import pandas as pd
import numpy as np
import requests

In [16]:
df_anuncios = pd.DataFrame()
url = 'https://www.milanuncios.com/motos-de-carretera/duke-390.htm'
page = requests.get(url, headers = headers)
page
soup = BeautifulSoup(page.content, 'html.parser')

In [17]:
# Encontramos los anuncios y los inspeccionamos. No podemos coger los primeros niveles tras el cuerpo, pero justo en el nivel 
# inferior hay etiquetas que parecen más generales, así que las usamos.

clase_anuncio ='aditem-detail-image-container' # Hemos cogido uno que no es del todo general, un poco por las prisas.
 
anuncios = soup.find_all('div', class_= clase_anuncio)
primer_anuncio = anuncios[0]
primer_anuncio.find('div', class_ = 'aditem-price').get_text() # Vamos inspeccionando que clase corresponde al precio.
primer_anuncio.find('a', class_ = 'aditem-detail-title').get_text() # Igual con el título del anuncio.
primer_anuncio.find('div', class_='ano tag-mobile').get_text() # Año

'año 2019'

In [18]:
for elemento in anuncios:
    Titulo = elemento.find('a', class_='aditem-detail-title').get_text()
    Precio = elemento.find('div', class_='aditem-price').get_text()
    Ano = elemento.find('div', class_='ano tag-mobile').get_text() # Nos da un error porque hay un dato vacío. 
                                                                   #Hay dos formas de resolverlo.
    print(Precio)

4.800€
3.500€
4.500€
3.850€


AttributeError: 'NoneType' object has no attribute 'get_text'

In [19]:
for elemento in anuncios:

 # titulo = elemento.find('a', class_='aditem-detail-title').get_text()

 # precio = elemento.find('div', class_='aditem-price').get_text()

  year = elemento.find('div', class_='ano tag-mobile')

  if year is not None:

    year = year.get_text()

  else:

    year = np.nan

  print(year)

# Vemos que había dos años vacíos (donde no se especificaba el año)

año 2019
año 2014
año 2019
año 2016
nan
año 2015
año 2006
año 2015
año 2016
año 2015
año 2016
año 2018
año 2017
año 2017
año 2014
año 2015
año 2018
año 2016
año 2009
año 2014
año 2020
año 2016
año 2014
año 2018
año 2013
año 2015
año 2018
año 2016
año 2014


In [20]:
df_anuncios = pd.DataFrame() # Para que siempre se reinicie el dataframe si resulta que le damos dos veces al for sin querer,
# Si no hiciéramos esto, la segunda vez se volverían a añadir los datos a la tabla por segunda vez.

for elemento in anuncios:
    fila = pd.Series() # Un panda series es una fila con un índice y un objeto, se podría hacer también con un pandas df, pero 
                       # podría dar algún problema.

    titulo = elemento.find('a', class_='aditem-detail-title').get_text()

    precio = elemento.find('div', class_='aditem-price').get_text()

    year = elemento.find('div', class_='ano tag-mobile')

    if year is not None:

      year = year.get_text()

    else:

      year = np.nan
        
    fila['titulo_anuncio'] = titulo
    fila['precio'] = precio
    fila['anyo'] = year
    
    # Solo queda ir metiendo esto como filas al dataframe de anuncios.
    
    df_anuncios =df_anuncios.append(fila,ignore_index=True)

df_anuncios 

    
    

<ipython-input-20-8eafca74a7b6>:5: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  fila = pd.Series() # Un panda series es una fila con un índice y un objeto, se podría hacer también con un pandas df, pero


,anyo,precio,titulo_anuncio
0,año 2019,4.800€,KTM - 390
1,año 2014,3.500€,KTM - KTM DUKE 390
2,año 2019,4.500€,KTM - DUKE
3,año 2016,3.850€,KTM - DUKE 390 ABS
4,NaN,1.000€,SE COMPRAN MOTOCICLETAS HONDA YAMAHA KAW
5,año 2015,3.600€,KTM - DUKE 390
6,año 2006,2.900€,SUZUKI - GSR 600 NAKED
7,año 2015,3.599€,KTM - DUKE 390 ABS
8,año 2016,2.800€,KTM - DUKE 390
9,año 2015,3.400€,KTM - DUKE 390


"Ahora ponedme los precios como floats."

In [21]:
df_anuncios['precio'][0].replace('€','').replace('.','') # Ya sabemos como dejar los datos de tal forma que los podamos castear
# a floats.

'4800'

In [22]:
# Aplicándolo al dataframe completo:
df_anuncios['precio_num'] = df_anuncios.apply(lambda row: float(row['precio'].replace('€','').replace('.','')), axis=1)
# Aplicándolo solamente sobre la columna precio:
df_anuncios['precio_num'] = df_anuncios['precio'].apply(lambda x: float(x.replace('€','').replace('.','')))
# En este último caso, como actuamos sobre una serie, no tiene ejes (no ponemos axis).
# Comprobemos si el tipo es, en efecto, float:
df_anuncios.dtypes

anyo               object
precio             object
titulo_anuncio     object
precio_num        float64
dtype: object

### Ejercicio

Modifica el código anterior para que, además de bajarse la página actual, navegue por el resto de páginas e incorpore también esos anuncios a tu dataframe.

## Scraping de tablas

(__leer en casa__)

A menudo, la información que nos interesa descargar está en tablas y nuestro objetivo es importarlas en tablas de Pandas. Esta conversión suele exigir la manipulación del texto, números y fechas contenidas en la tabla original, lo que nos obligará a repasar cómo realizar esas operaciones y aplicarlas a filas y columnas de las tablas.

La estructura que suelen tener la tablas en `html` es:

```
<table>
    <thead>
        <tr>
            <th>Columna A</th>
            <th>Columna B</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>A1</td>
            <td>B1</td>
        </tr>
        <tr>
            <td>A2</td>
            <td>B2</td>
        </tr>
    </tbody>
</table>   
```

<table>
    <thead>
        <tr>
            <th>Columna A</th>
            <th>Columna B</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>A1</td>
            <td>B1</td>
        </tr>
        <tr>
            <td>A2</td>
            <td>B2</td>
        </tr>
    </tbody>
</table> 

Necesitaremos los siguientes módulos además de `requests` y `BeautifulSoup` importados anteriormente:

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

Primero, hacemos una petición para descargar la página de interés (que contiene las cotizaciones de las acciones del IBEX 35 en tiempo _casi_ real).

In [2]:
base_url = "https://www.eleconomista.es/indice/IBEX-35"
res = requests.get(base_url)
contenido = res.content

La siguiente línea procesa el HTML de la página que hemos descargado:

In [12]:
soup = BeautifulSoup(contenido, "html.parser")

Una vez procesado el HTML, es posible buscar elementos dentro de él. En particular, podemos buscar los elementos de tipo `table`, es decir, tablas.

In [13]:
tablas = soup.find_all('table') # Buscamos primero todas las tablas de la web, que llevan el tag <table>.

El objeto `tablas` contiene todas las tablas presentes en la página. Hay que tener cuidado con dichas tablas porque muchas páginas utilizan elementos de tipo `table` para estructurar el contenido. Por eso, en algunas páginas, aunque parezca haber una única tabla, puede haber otras con una información no interesante que toca descartar.

In [5]:
len(tablas) # Podemos comprobar que hemos traído solo 3 tablas, que es lo que nos interesa. Aunque para asegurar lo suyo es 
            # añadir una class_.

3

In [6]:
# Si quisiéramos solo la primera tabla, como ya podemos tratar el contenido como una lista, podemos hacer:

mi_tabla = tablas[0]

In [7]:
mi_tabla.find('thead') # Sabemos que solo hay un encabezado para cada tabla, así que usamos .find() para encontrarlo.

<thead><tr class="footable-header"><th class="footable-first-visible" href="/indice/IBEX-35/resumen/Nombre/descendente">Nombre</th><th>Precio</th><th data-breakpoints="" data-title=""></th><th data-breakpoints=""><a href="/indice/IBEX-35/resumen/Mejores">Var. (%)</a></th><th data-breakpoints="xs">Var. (€)</th><th data-breakpoints="sm xs" data-title="Volumen (€)"><a href="/indice/IBEX-35/resumen/Volumen">Volumen (€)</a></th><th data-breakpoints="sm xs" data-title="Cap."><a href="/indice/IBEX-35/resumen/Capitalizacion">Capitalización</a><sup>(1)</sup></th><th data-breakpoints="sm xs" data-title="PER"><a href="/indice/IBEX-35/resumen/PER">PER</a></th><th data-breakpoints="sm xs" data-title="Rent. /Div."><a href="/indice/IBEX-35/resumen/Rentabilidad-Dividendo">Rent. /Div.</a></th><th class="footable-last-visible" data-breakpoints="sm xs" data-title="Hora"> Hora </th></tr></thead>

In [9]:
mi_tabla.find('thead').find('tr').find_all('th') # Vamos filtrando hasta llegar al th, que es donde está la información.

# Realmente, el find('tr') aquí es prescindible, podemos saltar al 'th' directamente porque no hay varios en el 'tr'.

[<th class="footable-first-visible" href="/indice/IBEX-35/resumen/Nombre/descendente">Nombre</th>,
 <th>Precio</th>,
 <th data-breakpoints="" data-title=""></th>,
 <th data-breakpoints=""><a href="/indice/IBEX-35/resumen/Mejores">Var. (%)</a></th>,
 <th data-breakpoints="xs">Var. (€)</th>,
 <th data-breakpoints="sm xs" data-title="Volumen (€)"><a href="/indice/IBEX-35/resumen/Volumen">Volumen (€)</a></th>,
 <th data-breakpoints="sm xs" data-title="Cap."><a href="/indice/IBEX-35/resumen/Capitalizacion">Capitalización</a><sup>(1)</sup></th>,
 <th data-breakpoints="sm xs" data-title="PER"><a href="/indice/IBEX-35/resumen/PER">PER</a></th>,
 <th data-breakpoints="sm xs" data-title="Rent. /Div."><a href="/indice/IBEX-35/resumen/Rentabilidad-Dividendo">Rent. /Div.</a></th>,
 <th class="footable-last-visible" data-breakpoints="sm xs" data-title="Hora"> Hora </th>]

----------------- A PARTIR DE AQUÍ LO HECHO EN CLASE (EQUIVALENTE A LO QUE SE HACE MÁS TARDE) -------------------------------

In [47]:
list()

[]

In [46]:
col_df = list() # Es equivalente a escribir [].

for elemento in mi_tabla.find('thead').find_all('th'):
    col_df.append(elemento.get_text())
col_df

['Nombre',
 'Precio',
 '',
 'Var. (%)',
 'Var. (€)',
 'Volumen (€)',
 'Capitalización(1)',
 'PER',
 'Rent. /Div.',
 ' Hora ']

In [45]:
mi_tabla.find('tbody').find_all('tr')[0].find_all('td') 

# Hay un solo tbody por cada tabla (normalmente), luego, dentro de cada 
# 'tr' (table row) hay varios elementos que, si dejamos así, nos los va a dar todos juntos en un string por cada fila. Por lo 
# tanto, para cada fila hay que volver a hacer un find_all('td'), que nos saque todos los 'td' por cada fila (cada 'td' contiene
# un elemento de la fila, así que así estamos obteniendo un string para cada elemento de cada fila).

# Nos hace falta un bucle para ir ejecutando la línea sobre cada fila ('tr').

[<td class="footable-first-visible" itemscope="" itemtype="http://schema.org/SiteNavigationElement"><a href="/empresa/ACCIONA" itemprop="url">ACCIONA</a></td>,
 <td><span id="F|compo|item_100022786_55_df|precio_ultima_cotizacion|div">109,70</span></td>,
 <td><span class="accion1" id="F|compo|item_100022786_55_df|#arrow|div"><img src="//s03.s3c.es/imag3/iconos/svg/cotizaciones-sube.svg"/></span></td>,
 <td><span class="accion1" id="F|compo|item_100022786_55_df|variacion_porcentual|div">+0,37%</span></td>,
 <td><span class="accion1" id="F|compo|item_100022786_55_df|variacion_puntos|div">0,40</span></td>,
 <td><span>11.564.875,70</span></td>,
 <td>0,00</td>,
 <td>33,29</td>,
 <td>1,80%</td>,
 <td><span class="footable-last-visible">16:28</span></td>]

In [52]:
filas_df = list()

for fila in mi_tabla.find('tbody').find_all('tr'):
    fila_actual = list()
    for elemento in fila.find_all('td'):
        fila_actual.append(elemento.get_text())
    filas_df.append(fila_actual)
        
filas_df
# 4ptos LEER, MOD, df + df WS (JOINs)
# 4ptos df pd -> Operaciones con matrices. Gestión de excepciones (Try).
# 3ptos Llamar API y trabajar con eso.
# Suma 11.

[['ACCIONA',
  '109,70',
  '',
  '+0,37%',
  '0,40',
  '11.564.875,70',
  '0,00',
  '33,29',
  '1,80%',
  '16:28'],
 ['ACERINOX',
  '9,02',
  '',
  '+2,99%',
  '0,26',
  '10.584.347,42',
  '0,00',
  '38,38',
  '5,73%',
  '16:28'],
 ['ACS',
  '26,87',
  '',
  '+1,32%',
  '0,35',
  '9.670.958,24',
  '0,00',
  '13,27',
  '5,90%',
  '16:28'],
 ['AENA',
  '136,20',
  '',
  '+0,22%',
  '0,30',
  '8.129.301,00',
  '0,00',
  '14,16',
  '0,00%',
  '16:28'],
 ['ALMIRALL',
  '10,96',
  '',
  '+1,48%',
  '0,16',
  '5.813.591,60',
  '0,00',
  '22,79',
  '1,82%',
  '16:27'],
 ['AMADEUS',
  '58,94',
  '',
  '-1,64%',
  '-0,98',
  '25.791.824,38',
  '0,00',
  '20,41',
  '0,00%',
  '16:27'],
 ['ARCELORMITTAL',
  '18,33',
  '',
  '+5,50%',
  '0,96',
  '13.917.643,30',
  '0,00',
  '0,00',
  '0,00%',
  '16:27'],
 ['BANKIA',
  '1,55',
  '',
  '+1,98%',
  '0,03',
  '3.873.744,99',
  '0,00',
  '53,19',
  '0,00%',
  '16:28'],
 ['BANKINTER',
  '4,47',
  '',
  '+1,00%',
  '0,04',
  '38.707.958,57',
  '0,00',
  

In [54]:
df_Datos = pd.DataFrame(filas_df, columns=col_df)
df_Datos

,Nombre,Precio,,Var. (%),Var. (€),Volumen (€),Capitalización(1),PER,Rent. /Div.,Hora
0,ACCIONA,"109,70",,"+0,37%","0,40","11.564.875,70","0,00","33,29","1,80%",16:28
1,ACERINOX,"9,02",,"+2,99%","0,26","10.584.347,42","0,00","38,38","5,73%",16:28
2,ACS,"26,87",,"+1,32%","0,35","9.670.958,24","0,00","13,27","5,90%",16:28
3,AENA,"136,20",,"+0,22%","0,30","8.129.301,00","0,00","14,16","0,00%",16:28
4,ALMIRALL,"10,96",,"+1,48%","0,16","5.813.591,60","0,00","22,79","1,82%",16:27
5,AMADEUS,"58,94",,"-1,64%","-0,98","25.791.824,38","0,00","20,41","0,00%",16:27
6,ARCELORMITTAL,"18,33",,"+5,50%","0,96","13.917.643,30","0,00","0,00","0,00%",16:27
7,BANKIA,"1,55",,"+1,98%","0,03","3.873.744,99","0,00","53,19","0,00%",16:28
8,BANKINTER,"4,47",,"+1,00%","0,04","38.707.958,57","0,00","15,05","2,06%",16:28
9,BBVA,"4,06",,"+0,87%","0,04","31.105.922,48","0,00","10,18","0,76%",16:28


In [62]:
# Otra forma:

df_Datos2 = pd.DataFrame(filas_df)
df_Datos2.columns = col_df
df_Datos2

,Nombre,Precio,,Var. (%),Var. (€),Volumen (€),Capitalización(1),PER,Rent. /Div.,Hora
0,ACCIONA,"109,70",,"+0,37%","0,40","11.564.875,70","0,00","33,29","1,80%",16:28
1,ACERINOX,"9,02",,"+2,99%","0,26","10.584.347,42","0,00","38,38","5,73%",16:28
2,ACS,"26,87",,"+1,32%","0,35","9.670.958,24","0,00","13,27","5,90%",16:28
3,AENA,"136,20",,"+0,22%","0,30","8.129.301,00","0,00","14,16","0,00%",16:28
4,ALMIRALL,"10,96",,"+1,48%","0,16","5.813.591,60","0,00","22,79","1,82%",16:27
5,AMADEUS,"58,94",,"-1,64%","-0,98","25.791.824,38","0,00","20,41","0,00%",16:27
6,ARCELORMITTAL,"18,33",,"+5,50%","0,96","13.917.643,30","0,00","0,00","0,00%",16:27
7,BANKIA,"1,55",,"+1,98%","0,03","3.873.744,99","0,00","53,19","0,00%",16:28
8,BANKINTER,"4,47",,"+1,00%","0,04","38.707.958,57","0,00","15,05","2,06%",16:28
9,BBVA,"4,06",,"+0,87%","0,04","31.105.922,48","0,00","10,18","0,76%",16:28


----------------- A PARTIR DE AQUÍ CONTINÚA LO QUE VENÍA HECHO DESDE EL PRINCIPIO EN EL NOTEBOOK -------------------------------

Podemos extraer las filas de todas estas tablas

In [ ]:
# todas_las_lineas = []
# for tabla in tablas:
#     lineas_tabla = tabla.find_all('tr') # Dentro de cada una de las 3 tablas, va a encontrar las filas (contienen el tab 'tr')
#     for x in lineas_tabla:              # Cada fila que haya encontrado en cada tabla...
#         todas_las_lineas.append(x)      # es añadida a la lista vacía creada al principio.

In [7]:
lineas = [x for tabla in tablas for x in tabla.find_all('tr')] # Forma simplificada. 

para luego extraer los contenidos de cada fila individualmente haciendo

In [ ]:
# datos=[]
# for linea in lineas:
#     linea_datos =[]
#     for x in linea.find_all('td'):
#         linea_datos.append(x.get_text())
#     datos.append(linea_datos)

In [8]:
datos = [[x.text for x  in linea.find_all('td')] for linea in lineas] # Forma simplificada.Es como hacer el bucle de dcha. a izq.

Podemos inspeccionar parte del objeto resultante:

In [9]:
datos[0:3]

[[],
 ['ACCIONA',
  '108,70',
  '',
  '-3,81%',
  '-4,30',
  '256.057.352,65',
  '0,00',
  '32,77',
  '1,83%',
  '11/12'],
 ['ACERINOX',
  '8,72',
  '',
  '-0,73%',
  '-0,06',
  '4.965.966,67',
  '0,00',
  '39,62',
  '5,55%',
  '11/12']]

Vemos que hay filas que contienen la información de interés junto con otras que contienen cabeceras y otra información irrelevante. En general, la situación puede ser más complicada y se hace necesario estudiar el objeto `tablas` para seleccionar la de interés.

En nuestro caso, podemos filtrar las líneas menos relevantes así:

In [10]:
datos = [x for x in datos if len(x) > 0]

Finalmente, podemos crear una tabla de Pandas:

In [11]:
datos = pd.DataFrame(datos)
datos

,0,1,2,3,4,5,6,7,8,9
0,ACCIONA,"108,70",,"-3,81%","-4,30","256.057.352,65","0,00","32,77","1,83%",11/12
1,ACERINOX,"8,72",,"-0,73%","-0,06","4.965.966,67","0,00","39,62","5,55%",11/12
2,ACS,"26,68",,"+0,57%","0,15","16.770.073,52","0,00","13,69","5,72%",11/12
3,AENA,"136,10",,"0,00%","0,00","53.106.925,10","0,00","14,76","0,00%",11/12
4,ALMIRALL,"10,71",,"-0,74%","-0,08","3.280.992,98","0,00","22,77","1,82%",11/12
5,AMADEUS,"60,22",,"-1,60%","-0,98","47.875.274,20","0,00","21,69","0,00%",11/12
6,ARCELORMITTAL,"17,19",,"-1,37%","-0,24","6.467.938,75","0,00","0,00","0,00%",11/12
7,BANKIA,"1,50",,"-1,64%","-0,03","5.878.101,65","0,00","54,15","0,00%",11/12
8,BANKINTER,"4,26",,"-2,38%","-0,10","16.062.250,86","0,00","15,81","1,96%",11/12
9,BBVA,"3,97",,"-2,05%","-0,08","164.534.116,93","0,00","11,04","0,70%",11/12


#### Ejercicio

Usa los elementos `th` de la primera fila de las tablas para extraer nombres para las columnas de la tabla. 

#### Ejercicio

Elimina las columnas irrelevantes y cambia los nombres de las columnas por otros breves y sin caracteres extraños o que dificulten el posproceso.

#### Ejercicio

Cambia el formato de las columnas adecuadamente: convierte a numéricas, etc., las columnas que lo requieran.

## Riesgos del scraping

El scraping es una técnica potente pero tiene varios contras:

* Implica mayor tiempo de desarrollo y mayor esfuerzo en la limpieza de datos (en comparación con otras fuentes como APIs, BDs, ...)
* Si hay que scrapear gran cantidad de páginas, es lento
* Los servidores objetivo de nuestro scraping pueden tener técnicas para evitarlo. Por ejemplo, bloquear la IP temporalmente o introducir delays en las respuestas si hacemos muchas peticiones en poco tiempo. Esto pasa especialmente en las grandes webs recelosas de sus datos (p.e. linkedin, amazon, ...).
* El código de scraping escrito hoy puede no funcionar mañana, si la web destino cambia nombres, etiquetas o estructura. Si se sube a producción para lanzarlo periódicamente, hay que ser conscientes de que en algún momento fallará, y establecer mecanismos de alerta

## Javascript

Es posible que te encuentres con algún caso en el que no puedas descargar tal cual el html y parsearlo, principalmente por dos motivos:

* La estructura de la página se genera parcial o totalmente en cliente
* Debemos interactuar con algún elemento para mostrar la información que queremos (p.e. completar un campo de búsuqeda, hacer click en algún botón, ...)

En estos casos, hay que ejecutar en un navegador local el código javascript de la página destino. Para esta tarea, puedes utilizar Selenium.

[Aquí](https://medium.freecodecamp.org/better-web-scraping-in-python-with-selenium-beautiful-soup-and-pandas-d6390592e251) un post con un ejemplo de uso.